# OrthodoxAI - Agents

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

## Libraries

In [1]:
# Load the API Keys
import os
from dotenv import load_dotenv
_ = load_dotenv()

## OpenAI

In [82]:
from pydantic import BaseModel, Field
from typing import List, Literal

import openai
from openai import OpenAI

In [83]:
class AnalyzerOutput(BaseModel):
    classification: Literal["Religious", "Non-Religious"] = Field(..., description="Either 'Religious' or 'Non-Religious'")
    key_topics: List[str] = Field(..., description="List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",)
    context_requirements: str = Field(..., description="A clear explanation of the query's context needs")
    query_complexity: Literal["Low", "Medium", "High"] = Field(..., description="'Low', 'Medium', or 'High' complexity")
    reasoning: str = Field(..., description="The Chain of Thought that has been done in order to analyze the user query")

In [84]:
MODEL_NAME = 'o3-mini'

client = OpenAI()

chat_template = [
    {"role": "system", "content": "You are an AI assistant that classifies user queries as either religious or non-religious and extracts key topics."},
    {"role": "user", "content": "Helloo!!"}
]

In [87]:
with client.beta.chat.completions.stream(
    model=MODEL_NAME,
    messages=chat_template,
    response_format=AnalyzerOutput,
) as stream:
    for event in stream:
        if event.type == "content.delta":
            if event.parsed is not None:
                # Print the parsed data as JSON
                print("content.delta parsed:", event.parsed)
            elif event.type == "content.done":
                print("content.done")
            elif event.type == "error":
                print("Error in stream:", event.error)

final_completion = stream.get_final_completion()

content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {}
content.delta parsed: {'classification': 'Non-Religious'}
content.delta parsed: {'classification': 'Non-Religious'}
content.delta parsed: {'classification': 'Non-Religious'}
content.delta parsed: {'classification': 'Non-Religious'}
content.delta parsed: {'classification': 'Non-Religious'}
content.delta parsed: {'classification': 'Non-Religious'}
content.delta parsed: {'classification': 'Non-Religious', 'key_topics': []}
content.delta parsed: {'classification': 'Non-Religious', 'key_topics': []}
content.delta parsed: {'classification': 'Non-Religious', 'key_topics': []}
content.delta parsed: {'classification': 'Non-Religious', 'key_topics': ['greeting']}
content.delta parsed: {'classification': 'Non-Religious', 'key_topics': ['greeting']}
co

In [108]:
final_completion

ParsedChatCompletion[AnalyzerOutput](id='chatcmpl-BEkHTAgxQaDt3kKJtST4gIvZZlEpt', choices=[ParsedChoice[AnalyzerOutput](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[AnalyzerOutput](content='{\n  "classification": "Non-Religious",\n  "key_topics": ["greeting"],\n  "context_requirements": "The query is a simple greeting without additional context related to any specific topic. Further information would be necessary to determine if there\'s a deeper intent.",\n  "query_complexity": "Low",\n  "reasoning": "The user query \'Helloo!!\' is a casual greeting that does not reference or imply any religious context. Therefore, it is classified as non-religious with a basic context focused on greetings."\n}', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, parsed=AnalyzerOutput(classification='Non-Religious', key_topics=['greeting'], context_requirements="The query is a simple greeting without additional context related to any s

In [110]:
final_completion.choices[0].message.parsed

AnalyzerOutput(classification='Non-Religious', key_topics=['greeting'], context_requirements="The query is a simple greeting without additional context related to any specific topic. Further information would be necessary to determine if there's a deeper intent.", query_complexity='Low', reasoning="The user query 'Helloo!!' is a casual greeting that does not reference or imply any religious context. Therefore, it is classified as non-religious with a basic context focused on greetings.")

In [113]:
class GetWeather(BaseModel):
    city: str
    country: str

client = OpenAI()

with client.beta.chat.completions.stream(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "What's the weather like in SF and London?",
        },
    ],
    tools=[
        openai.pydantic_function_tool(GetWeather, name="get_weather"),
    ],
    parallel_tool_calls=True,
) as stream:
    for event in stream:
        if event.type == "tool_calls.function.arguments.delta" or event.type == "tool_calls.function.arguments.done":
            print(event)

# print(stream.get_final_completion())

FunctionToolCallArgumentsDeltaEvent(type='tool_calls.function.arguments.delta', name='get_weather', index=0, arguments='', parsed_arguments=None, arguments_delta='')
FunctionToolCallArgumentsDeltaEvent(type='tool_calls.function.arguments.delta', name='get_weather', index=0, arguments='{"ci', parsed_arguments={}, arguments_delta='{"ci')
FunctionToolCallArgumentsDeltaEvent(type='tool_calls.function.arguments.delta', name='get_weather', index=0, arguments='{"city": ', parsed_arguments={}, arguments_delta='ty": ')
FunctionToolCallArgumentsDeltaEvent(type='tool_calls.function.arguments.delta', name='get_weather', index=0, arguments='{"city": "San F', parsed_arguments={}, arguments_delta='"San F')
FunctionToolCallArgumentsDeltaEvent(type='tool_calls.function.arguments.delta', name='get_weather', index=0, arguments='{"city": "San Franc', parsed_arguments={}, arguments_delta='ranc')
FunctionToolCallArgumentsDeltaEvent(type='tool_calls.function.arguments.delta', name='get_weather', index=0, arg

In [114]:
stream.get_final_completion()

ParsedChatCompletion[NoneType](id='chatcmpl-BEkNyuTsSGiDjJ5deIB5AVxQmVPDZ', choices=[ParsedChoice[NoneType](finish_reason='tool_calls', index=0, logprobs=None, message=ParsedChatCompletionMessage[NoneType](content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ParsedFunctionToolCall(id='call_0mOyIlPaGumVDzjF41CGiUBP', function=ParsedFunction(arguments='{"city": "San Francisco", "country": "USA"}', name='get_weather', parsed_arguments=GetWeather(city='San Francisco', country='USA')), type='function', index=0), ParsedFunctionToolCall(id='call_sxEsxJW1dJDBpPx0gbaKca1D', function=ParsedFunction(arguments='{"city": "London", "country": "UK"}', name='get_weather', parsed_arguments=GetWeather(city='London', country='UK')), type='function', index=1)], parsed=None))], created=1742854598, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_c1e1ac6736', usage=None)

## Langchain LLM Chains

### Structure Outputs

In [80]:
from pydantic import BaseModel, Field
from typing import List, Literal

In [81]:
class AnalyzerOutput(BaseModel):
    classification: Literal["Religious", "Non-Religious"] = Field(..., description="Either 'Religious' or 'Non-Religious'")
    key_topics: List[str] = Field(..., description="List of key topics/areas related to the user's question (e.g., theology, jesus, humility, virtues)",)
    context_requirements: str = Field(..., description="A clear explanation of the query's context needs")
    query_complexity: Literal["Low", "Medium", "High"] = Field(..., description="'Low', 'Medium', or 'High' complexity")
    reasoning: str = Field(..., description="The Chain of Thought that has been done in order to analyze the user query")

### Analyzer Agent

In [78]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [70]:
analyzer_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an AI assistant that classifies user queries as either religious or non-religious and extracts key topics.",
            ),
            (
                "human",
                "Classify the following query and extract key concepts:\n\nQuery: {query}",
            ),
        ]
    )

In [69]:
analyzer_llm = ChatOpenAI(model="o3-mini").with_structured_output(AnalyzerOutput)

In [71]:
analyzer_chain = analyzer_prompt | analyzer_llm

In [72]:
text = analyzer_chain.invoke('Hello')
text

AnalyzerOutput(classification='Non-Religious', key_topics=['greeting'], context_requirements='No additional context is provided beyond a basic greeting.', query_complexity='Low', reasoning="The query 'Hello' is a simple greeting without any religious connotations. It is non-religious in nature and does not contain hints of theological or religious content, so it is classified as Non-Religious with key focus on greeting.")

In [73]:
print(text.classification)
print(text.context_requirements)
print(text.key_topics)
print(text.query_complexity)
print(text.reasoning)

Non-Religious
No additional context is provided beyond a basic greeting.
['greeting']
Low
The query 'Hello' is a simple greeting without any religious connotations. It is non-religious in nature and does not contain hints of theological or religious content, so it is classified as Non-Religious with key focus on greeting.


In [74]:
type(text)

__main__.AnalyzerOutput

In [77]:
analysis_report = (
    f"{text.reasoning}\n\n"
    f"The analysis of the user query shows that the question is {text.classification} "
    f"and with a {text.query_complexity} complexity. Specifically, the analysis said the following: "
    f"{text.context_requirements} Key topics: {', '.join(text.key_topics)}."
)
print(analysis_report)

The query 'Hello' is a simple greeting without any religious connotations. It is non-religious in nature and does not contain hints of theological or religious content, so it is classified as Non-Religious with key focus on greeting.

The analysis of the user query shows that the question is Non-Religious and with a Low complexity. Specifically, the analysis said the following: No additional context is provided beyond a basic greeting. Key topics: greeting.


### Summarizer Agent